# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import Point
from datetime import datetime
import geopandas as gpd
from geopandas import GeoDataFrame
import geodatasets
pd.set_option('display.max_columns', None)

# Train Test to Full

In [2]:
df = pd.read_parquet('../data/raw/train.parquet')

In [3]:
df_2 = pd.read_parquet('../data/raw/test.parquet')

In [4]:
df = pd.concat([df, df_2], ignore_index=True)

In [5]:
df.to_parquet('../data/raw/full.parquet.gzip', compression='gzip', index=False)

# Merging Data

In [2]:
df = pd.read_parquet('../data/raw/full.parquet.gzip')

In [3]:
# Weather key
df['year_month_day'] = df['tpep_pickup_datetime'].astype(str).str[0:10].str.replace('-', '')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,year_month_day
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,20240101
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,20240101
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,20240101
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,20240101
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,20240101


## Weather

In [5]:
df_weather = pd.read_parquet('../data/external/weather_unify.parquet.gzip')

In [6]:
df_weather['year_month_day'] = df_weather['Date'].apply(lambda x: datetime.strftime(datetime.strptime(x, '%d/%m/%Y'), '%Y%m%d'))

In [7]:
df_weather.head()

,Date,Maximum,Minimum,Average,Departure,HDD,CDD,Precipitation,new_snow,snow_depth,year_month_day
0,01/01/2024,45,32,38.5,5.4,26,0,0,0,0,20240101
1,02/01/2024,43,26,34.5,1.6,30,0,0,0,0,20240102
2,03/01/2024,45,28,36.5,3.8,28,0,0,0,0,20240103
3,04/01/2024,45,26,35.5,2.9,29,0,T,T,0,20240104
4,05/01/2024,38,21,29.5,-2.9,35,0,0,0,0,20240105


In [ ]:
df_weather.columns

Index(['Date', 'Maximum', 'Minimum', 'Average', 'Departure', 'HDD', 'CDD',
       'Precipitation', 'new_snow', 'snow_depth', 'year_month_day'],
      dtype='object')

In [9]:
df_interim = df.merge(df_weather[['year_month_day', 'Maximum', 'Minimum', 
                                  'Average', 'Precipitation', 'new_snow', 
                                  'snow_depth']], on='year_month_day', how='left')

In [10]:
df_interim.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,year_month_day,Maximum,Minimum,Average,Precipitation,new_snow,snow_depth
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0


In [ ]:
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 26 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

## Lat/Long Knowledge Base

In [12]:
df_lat_long = pd.read_csv('../data/external/taxi_zone_lookup.csv', delimiter=',')

In [13]:
df_lat_long.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [14]:
df_lat_long.columns

Index(['LocationID', 'Borough', 'Zone', 'service_zone'], dtype='object')

In [15]:
df_interim = df_interim.merge(df_lat_long, right_on='LocationID', left_on='PULocationID', how='left')

In [16]:
df_interim = df_interim.merge(df_lat_long, right_on='LocationID', 
                              left_on='DOLocationID', how='left', suffixes=('_PU', '_DO'))

# Drop Unused Columns

In [17]:
df_interim.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'year_month_day',
       'Maximum', 'Minimum', 'Average', 'Precipitation', 'new_snow',
       'snow_depth', 'LocationID_PU', 'Borough_PU', 'Zone_PU',
       'service_zone_PU', 'LocationID_DO', 'Borough_DO', 'Zone_DO',
       'service_zone_DO'],
      dtype='object')

In [18]:
df_interim.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,year_month_day,Maximum,Minimum,Average,Precipitation,new_snow,snow_depth,LocationID_PU,Borough_PU,Zone_PU,service_zone_PU,LocationID_DO,Borough_DO,Zone_DO,service_zone_DO
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone,79,Manhattan,East Village,Yellow Zone
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0,140,Manhattan,Lenox Hill East,Yellow Zone,236,Manhattan,Upper East Side North,Yellow Zone
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0,236,Manhattan,Upper East Side North,Yellow Zone,79,Manhattan,East Village,Yellow Zone
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0,79,Manhattan,East Village,Yellow Zone,211,Manhattan,SoHo,Yellow Zone
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,20240101,45.0,32.0,38.5,0,0,0,211,Manhattan,SoHo,Yellow Zone,148,Manhattan,Lower East Side,Yellow Zone


In [21]:
df_interim.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 34 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               2964624 non-null  int32         
 1   tpep_pickup_datetime   2964624 non-null  datetime64[us]
 2   tpep_dropoff_datetime  2964624 non-null  datetime64[us]
 3   passenger_count        2824462 non-null  float64       
 4   trip_distance          2964624 non-null  float64       
 5   RatecodeID             2824462 non-null  float64       
 6   store_and_fwd_flag     2824462 non-null  object        
 7   PULocationID           2964624 non-null  int32         
 8   DOLocationID           2964624 non-null  int32         
 9   payment_type           2964624 non-null  int64         
 10  fare_amount            2964624 non-null  float64       
 11  extra                  2964624 non-null  float64       
 12  mta_tax                29646

# Cast and Missing Input

In [38]:
cast_dict = {
    'int32' : ['vendor_id', 'passenger_count', 'year_month_day', 
             'max_temperature_normal_f', 'min_temperature_normal_f', 
             'avg_temperature_normal_f'],
    'float32' : ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
                 'dropoff_latitude', 'daily_preciptation_normal_inches'],
    'datetime[64]' : ['pickup_datetime']
}

In [42]:
class FixVariable():
    """Class to fix variable's type and missing values
    """

    def __init__(self, cast_dict):

        self.cast_dict = cast_dict

        self.type_function = {
            'float32' : np.float32,
            'float64' : np.float64,
            'str' : str,
            'int64' : np.int64,
            'int32' : np.int32,
            'datetime[64]' : 'datetime64[s]'
        }

    def fit(self, X, y=None):
        return self

    def fix_type(self, X):
        for type, columns in self.cast_dict.items():
            X[columns] = X[columns].astype(self.type_function[type])
        return X
    
    def fix_missing(self, X):
        return X

    def transform(self, X):
        X_tmp = X.reset_index(drop=True)
        X_tmp = self.fix_missing(X_tmp)
        X_tmp = self.fix_type(X_tmp)
        return X_tmp

In [43]:
fix_vars = FixVariable(cast_dict)

In [47]:
df_interim = fix_vars.transform(df_interim)

In [48]:
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624255 entries, 0 to 624254
Data columns (total 14 columns):
 #   Column                            Non-Null Count   Dtype        
---  ------                            --------------   -----        
 0   id                                624255 non-null  object       
 1   vendor_id                         624255 non-null  int32        
 2   pickup_datetime                   624255 non-null  datetime64[s]
 3   passenger_count                   624255 non-null  int32        
 4   pickup_longitude                  624255 non-null  float32      
 5   pickup_latitude                   624255 non-null  float32      
 6   dropoff_longitude                 624255 non-null  float32      
 7   dropoff_latitude                  624255 non-null  float32      
 8   store_and_fwd_flag                624255 non-null  object       
 9   year_month_day                    624255 non-null  int32        
 10  daily_preciptation_normal_inches  624255 non

In [49]:
df_interim.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,year_month_day,daily_preciptation_normal_inches,max_temperature_normal_f,min_temperature_normal_f,avg_temperature_normal_f
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,20160630,0.13,84,69,76
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,20160630,0.13,84,69,76
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,20160630,0.13,84,69,76
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,20160630,0.13,84,69,76
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,20160630,0.13,84,69,76


In [53]:
df_interim.to_parquet(
    '../data/interim/interim.parquet.gzip',
    compression='gzip',
    index=False
)